# Building a Logistic Regression model with Pyspark

Spark is a powerful, general purpose tool for working with Big Data. Spark transparently handles the distribution of compute tasks across a cluster. This means that operations are fast, but it also allows you to focus on the analysis rather than worry about technical details. You'll learn Logistic Regression/Classifiers. 

### Index 
1. Loading the data
2. Data Preparation
3. Train/test split
4. Build a Logistic Regression model
5. Evaluate the Logistic Regression

In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import SparkSession                     # Import the PySpark module
from pyspark.sql.functions import round                  # Import the required function
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression # Import the logistic regression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.feature import OneHotEncoderEstimator    # Import the one hot encoder class

In [2]:
# Create SparkSession object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('LogisticRegression') \
                    .getOrCreate()

## 1. Loading the data

In [3]:
path = "/home/danae/Documents/pySparkTraining/files/"

# Read data from CSV file
flights = spark.read.csv(path + 'flights.csv',
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA')

# Get number of records
print("The data contain %d records." % flights.count())
# Check column data types
print(flights.dtypes)
print(flights.count())

# View the first five records
flights.show(5)

The data contain 50000 records.
[('mon', 'int'), ('dom', 'int'), ('dow', 'int'), ('carrier', 'string'), ('flight', 'int'), ('org', 'string'), ('mile', 'int'), ('depart', 'double'), ('duration', 'int'), ('delay', 'int')]
50000
+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| null|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| null|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



## 2. Data Preparation
removing an uninformative column and
removing rows which do not have information about whether or not a flight was delayed.


In [4]:
# Remove the 'flight' column
flights_drop_column = flights.drop('flight')

# Number of records with missing 'delay' values
flights_drop_column.filter('delay IS NULL').count()

# Remove records with missing 'delay' values
flights_valid_delay = flights_drop_column.filter('delay IS NOT NULL')

# Remove records with missing values in any column and get the number of remaining rows
flights_none_missing = flights_valid_delay.dropna()
print(flights_none_missing.count())

flights_none_missing.show(5)

47022
+---+---+---+-------+---+----+------+--------+-----+
|mon|dom|dow|carrier|org|mile|depart|duration|delay|
+---+---+---+-------+---+----+------+--------+-----+
|  0| 22|  2|     UA|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|ORD|1236| 10.33|     195|   -5|
|  5|  2|  1|     UA|SFO| 550|  7.98|     102|    2|
|  7|  2|  6|     AA|ORD| 733| 10.83|     135|   54|
+---+---+---+-------+---+----+------+--------+-----+
only showing top 5 rows



## 2.1 Column manipulation

The Federal Aviation Administration (FAA) considers a flight to be "delayed" when it arrives 15 minutes or more after its scheduled time.

The next step of preparing the flight data has two parts:

1. convert the units of distance, replacing the mile column with a `kmcolumn`; and
2. create a Boolean column indicating whether or not a flight was delayed.

In [5]:
# Convert 'mile' to 'km' and drop 'mile' column
flights_km = flights_none_missing.withColumn('km', round(flights.mile * 1.60934 , 0)) \
                    .drop('mile')

# Create 'label' column indicating whether flight delayed (1) or not (0)
flights_km = flights_km.withColumn('label', (flights_km.delay >= 15).cast('integer'))

# Check first five records
flights_km.show(5)
print(flights_km.count())

+---+---+---+-------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|
+---+---+---+-------+---+------+--------+-----+------+-----+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|
+---+---+---+-------+---+------+--------+-----+------+-----+
only showing top 5 rows

47022


## 2.2 Categorical columns
In the flights data there are two columns, carrier and org, which hold categorical data. You need to transform those columns into indexed numerical values.

In [6]:
# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights_km)

# Indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights_km)
flights_indexed.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|        0.0|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|        0.0|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|        1.0|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|        0.0|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|        1.0|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+
only showing top 5 rows



In [7]:
# Repeat the process for the other categorical feature
flights_indexed = StringIndexer(inputCol='org', outputCol='org_idx')\
                                .fit(flights_indexed)\
                                .transform(flights_indexed)
flights_indexed.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|        0.0|    1.0|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|        1.0|    0.0|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|        0.0|    1.0|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|        1.0|    0.0|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
only showing top 5 rows



The first step to encoding categorical features is to create a `StringIndexer`. Members of this class are **Estimators** that take a DataFrame with a column of strings and map each unique string to a number. 

Then, the *Estimator* returns a **Transformer** that takes a DataFrame, attaches the mapping to it as metadata, and returns a new DataFrame with a numeric column corresponding to the string column.

In [8]:
flights_indexed.select('carrier', 'carrier_idx').distinct().orderBy('carrier_idx').show()
flights_indexed.select('org', 'org_idx').distinct().orderBy('org_idx').show()

+-------+-----------+
|carrier|carrier_idx|
+-------+-----------+
|     UA|        0.0|
|     AA|        1.0|
|     OO|        2.0|
|     WN|        3.0|
|     B6|        4.0|
|     OH|        5.0|
|     US|        6.0|
|     HA|        7.0|
|     AQ|        8.0|
+-------+-----------+

+---+-------+
|org|org_idx|
+---+-------+
|ORD|    0.0|
|SFO|    1.0|
|JFK|    2.0|
|LGA|    3.0|
|SMF|    4.0|
|SJC|    5.0|
|TUS|    6.0|
|OGG|    7.0|
+---+-------+



## 2.3 Onehot encoding

In [10]:
# Create an instance of the one hot encoder
flights_onehot = OneHotEncoderEstimator(inputCols=['org_idx'], outputCols=['org_dummy'])

# Apply the one hot encoder to the flights data
onehot = flights_onehot.fit(flights_indexed)
flights_onehot = onehot.transform(flights_indexed)

# Check the results
flights_onehot.select('org', 'org_idx', 'org_dummy').distinct().sort('org_idx').show()

+---+-------+-------------+
|org|org_idx|    org_dummy|
+---+-------+-------------+
|ORD|    0.0|(7,[0],[1.0])|
|SFO|    1.0|(7,[1],[1.0])|
|JFK|    2.0|(7,[2],[1.0])|
|LGA|    3.0|(7,[3],[1.0])|
|SMF|    4.0|(7,[4],[1.0])|
|SJC|    5.0|(7,[5],[1.0])|
|TUS|    6.0|(7,[6],[1.0])|
|OGG|    7.0|    (7,[],[])|
+---+-------+-------------+



In [11]:
# Repeat the process for the other categorical feature
flights_onehot = OneHotEncoderEstimator(inputCols=['carrier_idx']
                                        , outputCols=['carrier_dummy'])\
                .fit(flights_onehot)\
                .transform(flights_onehot)

# Check the results
flights_onehot.select('carrier', 'carrier_idx', 'carrier_dummy')\
    .distinct()\
    .sort('carrier_idx')\
    .show()

+-------+-----------+-------------+
|carrier|carrier_idx|carrier_dummy|
+-------+-----------+-------------+
|     UA|        0.0|(8,[0],[1.0])|
|     AA|        1.0|(8,[1],[1.0])|
|     OO|        2.0|(8,[2],[1.0])|
|     WN|        3.0|(8,[3],[1.0])|
|     B6|        4.0|(8,[4],[1.0])|
|     OH|        5.0|(8,[5],[1.0])|
|     US|        6.0|(8,[6],[1.0])|
|     HA|        7.0|(8,[7],[1.0])|
|     AQ|        8.0|    (8,[],[])|
+-------+-----------+-------------+



## 2.4 Assembling columns
The final stage of data preparation is to consolidate all of the predictor columns into a single column.

This has to be done before modeling because every Spark modeling routine expects the data to be in this form.

In [12]:
# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'mon', 'dom', 'dow', 'carrier_dummy', 'org_dummy', 'km', 'depart', 'duration'
], outputCol='features')

# Consolidate predictor columns
flights_assembled = assembler.transform(flights_onehot)

# Check the resulting column
flights_assembled.select('label', 'features').show(5, truncate = False)

+-----+--------------------------------------------------------------------+
|label|features                                                            |
+-----+--------------------------------------------------------------------+
|1    |(21,[1,2,3,11,18,19,20],[22.0,2.0,1.0,1.0,509.0,16.33,82.0])        |
|0    |(21,[0,1,2,3,12,18,19,20],[2.0,20.0,4.0,1.0,1.0,542.0,6.17,82.0])   |
|0    |(21,[0,1,2,4,11,18,19,20],[9.0,13.0,1.0,1.0,1.0,1989.0,10.33,195.0])|
|0    |(21,[0,1,2,3,12,18,19,20],[5.0,2.0,1.0,1.0,1.0,885.0,7.98,102.0])   |
|1    |(21,[0,1,2,4,11,18,19,20],[7.0,2.0,6.0,1.0,1.0,1180.0,10.83,135.0]) |
+-----+--------------------------------------------------------------------+
only showing top 5 rows



## 3. Train/test split

To objectively assess a Machine Learning model you need to be able to test it on an independent set of data. You can't use the same data that you used to train the model: of course the model will perform (relatively) well on those data!

You will split the data into two components:

- training data (used to train the model) and
- testing data (used to test the model).


In [13]:
# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights_assembled.randomSplit([0.8, 0.2], seed=17)

# Check that training set has around 80% of records
training_ratio = flights_train.count() / flights.count()
print(training_ratio)

0.75054


## 4. Build a Logistic Regression model
The objective is to predict whether a flight is likely to be delayed by at least 15 minutes (label 1) or not (label 0).

In [14]:
# Create a classifier object and train on training data
logistic = LogisticRegression().fit(flights_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(flights_test)

prediction.select('label', 'probability', 'prediction').show(5)

# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

+-----+--------------------+----------+
|label|         probability|prediction|
+-----+--------------------+----------+
|    1|[0.44630381940577...|       1.0|
|    1|[0.36353388740185...|       1.0|
|    1|[0.28861696644354...|       1.0|
|    1|[0.21152096951843...|       1.0|
|    1|[0.24782054530780...|       1.0|
+-----+--------------------+----------+
only showing top 5 rows

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1582|
|    0|       0.0| 2580|
|    1|       1.0| 3242|
|    0|       1.0| 2091|
+-----+----------+-----+



## 5. Evaluate the Logistic Regression model
Accuracy is generally not a very reliable metric because it can be biased by the most common target class.

There are two other useful metrics : precision and recall.

- **Precision** is the proportion of positive predictions which are correct. For all flights which are predicted to be delayed, what proportion is actually delayed?

- **Recall** is the proportion of positives outcomes which are correctly predicted. For all delayed flights, what proportion is correctly predicted by the model?

The precision and recall are generally formulated in terms of the positive target class. But it's also possible to calculate weighted versions of these metrics which look at both target classes.

In [15]:
# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN+TP)/(TN+TP+FN+FP)
print(accuracy)

0.6131648235913639


In [16]:
# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator =  MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: 
                                                           'weightedPrecision'})
# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName:'areaUnderROC'})

precision = 0.61
recall    = 0.67


In [17]:
# Terminate the cluster
spark.stop()